In [20]:
import cv2
import numpy as np

#何もしない関数を宣言
def nothing(x):
    pass

#ウィンドウに名前をつける
cv2.namedWindow('image')

#トラックバーを生成
cv2.createTrackbar('R', 'image', 0, 50, nothing)
cv2.createTrackbar('G', 'image', 0, 50, nothing)
cv2.createTrackbar('B', 'image', 0, 50, nothing)
cv2.createTrackbar('gamma', 'image', 0, 50, nothing)
cv2.createTrackbar('gaussian', 'image', 0, 1, nothing)
cv2.createTrackbar('filter_height', 'image', 0, 50, nothing)
cv2.createTrackbar('filter_width', 'image', 0, 50, nothing)
cv2.createTrackbar('sigma', 'image', 0, 100, nothing)
cv2.createTrackbar('reset', 'image', 0, 1, nothing)

#トラックバーの初期値を設定
cv2.setTrackbarPos('R','image', 10)
cv2.setTrackbarPos('G','image', 10)
cv2.setTrackbarPos('B','image', 10)
cv2.setTrackbarPos('gamma','image', 10)
cv2.setTrackbarPos('gaussian','image', 0)
cv2.setTrackbarPos('filter_height','image', 0)
cv2.setTrackbarPos('filter_width','image', 0)
cv2.setTrackbarPos('sigma','image', 1)
cv2.setTrackbarPos('reset','image', 0)


# VideoCaptureのインスタンスを作成する
# 引数でカメラを選択できる
cap = cv2.VideoCapture(0)

while True:
    # VideoCaptureから1フレーム読み込む
    ret, frame = cap.read()
    if not ret: continue
        
    # 1/3サイズに縮小
    frame = cv2.resize(frame, (int(frame.shape[1]/3), int(frame.shape[0]/3)))
    
    #赤色補正値を取得
    r = cv2.getTrackbarPos('R', 'image') * 0.1
    if r == 0:
        r = 0.1
        cv2.setTrackbarPos('R','image', 1)
        
    #赤色補正用ルックアップテーブル
    look_up_table = np.zeros((256, 1), dtype = 'uint8')
    for i in range(len(look_up_table)):
        look_up_table[i][0] = (len(look_up_table)-1) * pow(float(i) / (len(look_up_table)-1), 1 / r)
    
    #ルックアップテーブルによる赤色補正 
    cp_frame = frame
    frame = cv2.LUT(frame, look_up_table)
    frame[:,:,(0,1)] = cp_frame[:,:,(0,1)]
    
    #緑色補正値を取得
    g = cv2.getTrackbarPos('G', 'image') * 0.1
    if g == 0:
        g = 0.1
        cv2.setTrackbarPos('G','image', 1)
        
    #緑色補正用ルックアップテーブル
    look_up_table = np.zeros((256, 1), dtype = 'uint8')
    for i in range(len(look_up_table)):
        look_up_table[i][0] = (len(look_up_table)-1) * pow(float(i) / (len(look_up_table)-1), 1 / g)
    
    #ルックアップテーブルによる緑色補正 
    cp_frame = frame
    frame = cv2.LUT(frame, look_up_table)
    frame[:,:,(0,2)] = cp_frame[:,:,(0,2)]
 
    #青色補正値を取得
    b = cv2.getTrackbarPos('B', 'image') * 0.1
    if b == 0:
        b = 0.1
        cv2.setTrackbarPos('B','image', 1)
        
    #青色補正用ルックアップテーブル
    look_up_table = np.zeros((256, 1), dtype = 'uint8')
    for i in range(len(look_up_table)):
        look_up_table[i][0] = (len(look_up_table)-1) * pow(float(i) / (len(look_up_table)-1), 1 / b)
    
    #ルックアップテーブルによる青色補正 
    cp_frame = frame
    frame = cv2.LUT(frame, look_up_table)
    frame[:,:,(1,2)] = cp_frame[:,:,(1,2)]
    
    #ガンマ値取得
    gamma = cv2.getTrackbarPos('gamma', 'image') *0.1
    if gamma == 0:
        gamma = 0.1
        cv2.setTrackbarPos('gamma','image', 1)
        
    #ガンマ補正用ルックアップテーブル
    look_up_table = np.zeros((256, 1), dtype = 'uint8')
    for i in range(len(look_up_table)):
        look_up_table[i][0] = (len(look_up_table)-1) * pow(float(i) / (len(look_up_table)-1), 1 / gamma)
    
    #ルックアップテーブルによるガンマ補正 
    frame = cv2.LUT(frame, look_up_table)
    
    #ガウシアンフィルタ処理
    gaussian = cv2.getTrackbarPos('gaussian', 'image')
    if gaussian == 1:
        #filter_height,filter_widthの取得．偶数にならないように調節
        filter_height = cv2.getTrackbarPos('filter_height', 'image')
        if filter_height % 2 == 0:
            filter_height += 1
        filter_width = cv2.getTrackbarPos('filter_width', 'image')
        if filter_width % 2 == 0:
            filter_width += 1   
        #分散値の取得
        sigma = cv2.getTrackbarPos('sigma', 'image') * 0.1
        if sigma == 0:
            sigma = 0.1
            cv2.setTrackbarPos('sigma','image', 1)
        frame = cv2.GaussianBlur(frame, ksize=(filter_width,filter_height), sigmaX=sigma)
    
    reset = cv2.getTrackbarPos('reset', 'image')
    if reset == 1:
        cv2.setTrackbarPos('R','image', 10)
        cv2.setTrackbarPos('G','image', 10)
        cv2.setTrackbarPos('B','image', 10)
        cv2.setTrackbarPos('gamma','image', 10)
        cv2.setTrackbarPos('gaussian','image', 0)
        cv2.setTrackbarPos('filter_height','image', 0)
        cv2.setTrackbarPos('filter_width','image', 0)
        cv2.setTrackbarPos('sigma','image', 1)  
        cv2.setTrackbarPos('reset','image', 0)
    
    # 画像を表示
    cv2.imshow('image', frame)
    
    # キー入力を1ms待って,kが27(ESC)だったらbreakする
    k = cv2.waitKey(1)
    if k == 27:
        break
    
    
# キャプチャをリリースして，ウィンドウを全て閉じる
cap.release()
cv2.destroyAllWindows()